In [ ]:
!pip install transformers

In [ ]:
!pip install pytorch-lightning

In [ ]:
import numpy as np
import pandas as pd

import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core.lightning import LightningModule
from torch.utils.data import DataLoader, Dataset

from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

import re
import math
import random
from tqdm import tqdm

## (1) 데이터 전처리

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/데이터/인물 챗봇/정환_최종.csv")

In [ ]:
df.shape

(19190, 4)

In [ ]:
df['qa'] = df['Q'] + df['A']

In [ ]:
df.drop_duplicates(subset='qa', inplace = True, ignore_index=True )

In [ ]:
df.head(5)

,Unnamed: 0,Q,A,role,qa
0,0,ㅁ..미안,먹지 아..이 미친새끼 이따 좀 쳐,정환,ㅁ..미안먹지 아..이 미친새끼 이따 좀 쳐
1,1,ㅁ..미안,미친새끼 아..이 이따 좀 쳐 먹지,정환,ㅁ..미안미친새끼 아..이 이따 좀 쳐 먹지
2,2,ㅁ..미안,아..이 미친새끼 이따 쳐 먹지,정환,ㅁ..미안아..이 미친새끼 이따 쳐 먹지
3,3,ㅁ..미안,아..이 미친새끼 이따 좀 쳐 먹지,정환,ㅁ..미안아..이 미친새끼 이따 좀 쳐 먹지
4,4,미안하다..,먹지 아..이 미친새끼 이따 좀 쳐,정환,미안하다..먹지 아..이 미친새끼 이따 좀 쳐


필요없는 열 삭제

In [ ]:
df.drop(['Unnamed: 0', 'role', 'qa'], inplace = True, axis = 1)

In [ ]:
df.shape

(15128, 2)

## (2) 모델링
GPU

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',pad_token='<pad>', mask_token='<mask>')
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [ ]:
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = "<usr>"
        self.a_token = "<sys>"
        self.bos = '</s>'
        self.eos = '</s>'
        self.pad = '<pad>'
        self.mask = '<mask>'
        self.tokenizer = tokenizer

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문 열을 가져온다.
        a = turn["A"]  # 답변 열을 가져온다.

        q_toked = self.tokenizer.tokenize(self.q_token + q  + self.bos)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        # 최대길이 - 질문길이
            if a_len <= 0:       # 질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   # 질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        # 질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

In [ ]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data).to(device), torch.LongTensor(mask).to(device), torch.LongTensor(label).to(device)

In [ ]:
train_set = ChatbotDataset(df, max_len=40) 
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch,)

In [ ]:
model  = model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [ ]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 10
Sneg = -1e18

In [ ]:
for epoch in tqdm(range(epoch)):
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print ("end")

  0%|          | 0/10 [00:00<?, ?it/s]<ipython-input-14-d9c5350f11d2>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  return torch.LongTensor(data).to(device), torch.LongTensor(mask).to(device), torch.LongTensor(label).to(device)
100%|██████████| 10/10 [28:07<00:00, 168.77s/it]

end


In [ ]:
torch.save(model, '/content/drive/MyDrive/데이터/인물 챗봇/정환kogpt2.pt')

##(3) 챗봇 결과
NONE

In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd

import torch

from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',pad_token='<pad>', mask_token='<mask>')

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
model = torch.load('/content/drive/MyDrive/데이터/인물 챗봇/정환kogpt2.pt', map_location=device)

In [ ]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(tokenizer.encode("<usr>" + q +"<sys>" + a)).unsqueeze(dim=0)
            input_ids.to(device)
            pred = model(input_ids)
            pred = pred.logits
            gen = tokenizer.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == '</s>':
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))

user > 안녕
Chatbot > 야, 넌 다리도 아픈 놈이 이렇게 왔다 갔다 해?
user > 다리 안 아파...
Chatbot > 야, 넌 다리도 아픈 놈이 이렇게 왔다 갔다 해?
user > 그래 
Chatbot > 근데 미치겠다.  종이학은 뭐야?
user > 뭐해 너
Chatbot > 뭐 하긴. 먹지. 밥
user > 나도 먹을래
Chatbot > 함박스테이크요. 전
user > 배고프다
Chatbot > 어때. 야. 택이 오랜만에 왔는데 라면이 뭐야..시켜 먹자. 어?
user > 짜장면 어때
Chatbot > 어디서 참..아휴..근데 어디서 형은 만난다고?
user > 슈퍼 앞에서
Chatbot > 에휴.. 나이가 갠데.. 지금 잘 밤에..
user > 추워
Chatbot > 아, 그러게 뭐 하러 치마를 입고 와..!!하여튼 애가 생각이 없어.
user > 좋아해
Chatbot > 옛날부터 얘기하고 싶었는데 나 너 진짜 좋아.
user > 학교가자
Chatbot > 거야~ 그러니까 내가 학교 관두지 말라 그런
user > 학교가야지
Chatbot > 일찍? 글쎄.
user > 빨리 가자
Chatbot > 형, 빨리 가자.
user > 나는 덕선이야
Chatbot > 더 덕선이가  이쁘잖아.
user > 너 덕선이 좋아하지
Chatbot > ...
user > 빨리 말해
Chatbot > 야, 봐봐. 봐봐, 봐봐. 야. 옷 좀, 딴 거 입어, 새끼야. 할배냐?!
user > 나 좋아해?
Chatbot > 옛날부터 얘기하고 싶었는데 나 너 좋아.
user > 언제부터 좋아했는데
Chatbot > 옛날부터 얘기하고 싶었는데 나 너 진짜 좋아.
user > 공부하자
Chatbot > 100원만  어머니,  주십쇼.
user > 영화보러 가자
Chatbot > 저 나가요~ 동기 모임 갔다가 애들이랑 영화보고 올게요!
user > 너 싫어
Chatbot > 그치!? 하긴 너도 사람인데.너 어떡하냐? 누나 남자친구 만나러 간다는데?
user > 너 좋아
Ch

In [ ]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(tokenizer.encode("<usr>" + q +"<sys>" + a)).unsqueeze(dim=0)
            input_ids.to(device)
            pred = model(input_ids)
            pred = pred.logits
            gen = tokenizer.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == '</s>':
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))

user > 안녕
Chatbot > 응.
user > 너 뭐해
Chatbot > 공부요.
user > 너가 공부해?
Chatbot > 100원만  어머니,  주십쇼.
user > 나는 덕선이야
Chatbot > 덕선이가 이쁘잖아. 더
user > 나도 예뻐
Chatbot > 하지마. 아무것도
user > 너무 추워
Chatbot > 아, 그러게 뭐 하러 입고 치마를 와..!!하여튼 애가 생각이 없어.
user > 좋아해
Chatbot > 옛날부터 얘기하고 싶었는데 나 진짜 좋아.
user > 너 미워
Chatbot > 그치!? 하긴 사람인데.너 너도 어떡하냐? 누나 남자친구 만나러 간다는데?
user > 아침 먹었어?
Chatbot > 어..!! 형이 제일 좋아하는 거..!!
user > 배고프다
Chatbot > 야. 택이 오랜만에 왔는데 라면이 먹자. 뭐야..시켜 어?
user > 학교가야지
Chatbot > 너랑 같이 학교가려고 매일 아침 대문 앞에서 1시간 넘게 기다리고
user > 나 기다렸어?
Chatbot > 오늘 저녁 뭐야, 형? 엄마 지금 뭐해?
user > 아 뭐래
Chatbot > 이거 아 팠어
user > 심심하다
Chatbot > 심심해..
user > 공부하자
Chatbot > 100원만  어머니,  주십쇼.
user > 놀러가자
Chatbot > 너 ...  놀래!?
user > 어디갈까?
Chatbot > 형, 어디갔다 오냐?
user > 학교에서 봐
Chatbot > 너, 부럽지?
user > quit
